In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import math
import random
import matplotlib.pyplot as plt
from scipy import stats

import json
from datetime import timedelta, date, datetime
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
path = os.getcwd() + '/mfp-diaries.tsv'
data = pd.read_csv(path, sep='\t')
data

 მონაცემების საკმაოდ მოცულობითია 587186 სტრიქონი 4 სვეტზე. თითოეული სტიქონისთვის გვაქს აიდი, რომელიც იუზერის მაინდიფიცირებელი რიცხვია(1) , გვაქვს თარიღი(2), რომელ დღესაც იუზერმა შეიყვანა აპლიკაციაში მონაცემები. გვაქვს თვითონ მონაცემები , რომელიც იუზერმა შეიყვანა (3) . ეს მონაცემები დიქშენერების მასივი , რომელშიცი ისევ დიქშენერებია. თითოეულ ელემენტს აქვს საჭმლის სახელი, შემცველი ნუტრიენტები(ცილები , ცხიმები , ნახშირწყლები ... ).მეოთხე სვეტში არის ტოტალური რაოდენობა ნუტრიენტების და მიზანი . ანუ რა მიზანი ჰქონდა დასახული აპლიკაციას და რა რაოდენობის ნუტრიენტები მოიხმარა მომხმარებელმა . ერთი შეხედვით მონაცემები ისეთი სახითაა წარმოდგენილი , რომ სავარაუდოდ ანომალიათა და დივერგენციების რაოდენობა საკმაოდ იქნება . 
### დავიწყოთ ამ მონაცემების დაპარსვა,  და მათი გასუფთავება  .

შევხედოთ დროის რა მონაკვეთში არის მონაცემები ასახული .

In [ ]:
print("First Date: ", data[data.columns[1]].unique().min())
print("Last Date: ",data[data.columns[1]].unique().max())

შევხედოთ რამდენი განსხვავებული თარიღი გვხვდება

In [ ]:
uniques = data[data.columns[1]].unique()
print("unique dates: ", len(uniques))
start = datetime.strptime('2014-09-14','%Y-%m-%d').date()
end = datetime.strptime('2015-04-09','%Y-%m-%d').date()

როგორც ვხედავთ თარიღების დუპლიკაცია არაა რადგან 2015-04-09 - 2014-09-14 = 208 დღეს
ვნახოთ როგორი სიხშირით იყენებდა ხალხი აპლიკაციას. ეს საშუალებას მოგვცემს განვსაზღვროთ როგორი დინამიკა იყო ამ დროის პერიოდის განმავლობაში.

In [ ]:
user_activity = np.zeros(data[data.columns[0]].unique().max())
for i in range( int((end-start).days)+1):
    dat = start+timedelta(i)
    dat_str = dat.strftime('%Y-%m-%d')
    curr_dt = data[data[data.columns[1]]==dat_str]
    for j in range(curr_dt.shape[0]):
        user_activity[curr_dt.iloc[j,0]-1]+=1


In [ ]:
plt.hist(user_activity, bins=50)
plt.ylabel('Frequency')
plt.title('Frequency of User Activity')
plt.show()

როგორც ვხედავთ იუზერების აქტივობა დროის პერიოდის დასაწყისში ბევრად უფრო მეტი იყო და დროთა განმავლობაში დაიკლო ხოლო ბოლოს ისევ მოიმტა. სავარაუდოდ მსგავსი დინამიკა დამოკიდბულია აპლიკაციის გამოსვლის და მისი აქტუალურობის ასპქტთან . ამასთანავე დროის პერიოდი როდესაც ყველაზე მაღალია აქტივობა არის შემოდგომის პერიოდი. ეს მონაცემიც შეგვიძლია გამოვიყენოთ იმის გასაგებად , თუ რა დროს არის ყველაზე მეტი სარგებლის მომტანი აპლიკაციის გაპიარება . 

როგორც ვნახეთ ზევით , თითოეული საჭმელი შეიცავს გარკვეულ ნუტრიენტებს . ჩვენთვის საინტერესო გახდა რა 
ნუტრინენტებია მონაცემებში და რა რაოდენობით . სავარაუდოდ ზოგიერთი ნუტრიენტი ნაკლებად იქნება , ზოგიერთი მეტი,
რაც  მსგავსი ნუტრიენტების გადაგდების საშუალებას მოგვცემს

თითოეულ საჭმელს აქვს dishes პარამეტრი და თითოეულ dish_ში არის ნუტრიენტების მასივი . თითოეულ ნუტრიენტს აქვს სახელი და მისი შესაბამისი რაოდენობა . პრიველ ჯერზე დავთვალოთ რამდენჯერ გვხვდება თითოეული ნუტრიენტი მონაცემებში.

In [ ]:
unique_nutritions = {}
for i in range(data.shape[0]):
    foods = json.loads(data.iloc[i,2])
    for j in range(len(foods)):
        dishes = foods[j]['dishes']
        
        for curr_dish in range(len(dishes)):
            nutritions = dishes[curr_dish]['nutritions']
            
            for n in range(len(nutritions)):
                prto_type = nutritions[n]['name']
                if prto_type in unique_nutritions.keys():
                    unique_nutritions[prto_type]+=1
                else:
                    unique_nutritions[prto_type] = 1
                

ჯერ ვნახოთ რამდენი სახის ნუტრიენტები გვაქვს 

In [ ]:
print(unique_nutritions.keys())
print("Num of nutrient types: "+str(len(unique_nutritions.keys())))

ჩვენი პირველი ვარაუდი იყო რომ მაქსიმუმ 5 მახასიათებელი იქნებოდა , თუმცა როგორც ვხედავთ 17 სხვადასხვა სახის ნუტრიენტი გვაქვს

In [ ]:
sortedNutritions = sorted(unique_nutritions.items(), key=lambda x : x[1], reverse = True)
print(sortedNutritions)
sortedDictionary = {}
for nut in sortedNutritions:
    sortedDictionary[nut[0]] = nut[1]
print(sortedDictionary)

In [ ]:
plt.figure(figsize=(30,8))
plt.bar(x = sortedDictionary.keys(), height = unique_nutritions.values())
plt.title('Number of occurrences for each Nutrition')
plt.show()

ნუტრიენტებში ლიდერობენ კალორიები , პროტეინები , ნახშირწყლები,ცხიმები , შაქარი , ყველაზე მცირე რაოდენობით არის Vit C, Trn Fat,Ply Fat,Mon Fat,Vit A . 
ვინაიდან მათი რაოდენობა ძალიან მცირეა , გადავწყვიტეთ საერთოდ გადაგვეყარა.


In [ ]:
filteredNutritions = {}
not_needed_prots =['Vit A', 'Vit C', 'Trn Fat', 'Potass.',
                    'Mon Fat', 'Ply Fat','Iron','Calcium','Chol','Sat Fat']
for nutritionName in sortedDictionary.keys():
    if nutritionName not in not_needed_prots:
        filteredNutritions[nutritionName] = sortedDictionary[nutritionName]
print(filteredNutritions)

In [ ]:
plt.figure(figsize=(20,8))
plt.pie(filteredNutritions.values(), labels = filteredNutritions.keys(),autopct = "%1.1f%%")
plt.show()

დავთვალოთ საჭმელების უნიკალური მახასიათებლები 2 პარამეტრად, 
1.მომხმარებელს რამდენი აქვს შეყვანილი(რამდენი ჭამა)
2.მიზანი (აპლიკაციამ რამდენი დაუნიშნა)
ვნახოთ არის თუ არა დივერგენცია მონაცემებში და ნამდვილად არის თუ არა ტოტალები ტოლი ცალკეული ნუტრიენტიების ჯამების. 


როგორც აღვნიშნეთ 5 ტიპის ნუტრიენტი გადავაგდეთ, რადგან მათი მაჩვენებლი მნიშვნელოვნად მცირე იყო დანარჩენ ნუტრიენტებთან შედარებით . შესაბამისად დათვლებს მხოლოდ prot_names_ში არსებულ ნუტრიენტებზე გავაკეთებთ.


In [ ]:
prot_names = ['Calories', 'Carbs', 'Fat', 'Protein', 'Sodium', 'Sugar', 'Fiber']

ამ ეტაპზე ჩვენი მიზანია ვნახოთ რამდენად სწორია მონაცემები . დავთვალოთ თითოეულ სტრიქონში არსებული ნუტრიენტებისთვის ჯამები თუ ემთხვევა ტოტალების სვეტში არსებულ მნიშვნელობებს . ყოველი დაუმთხვეველი ტიპი ჩავყაროთ მასივში და ვნახოთ რა დივერგენცია ხდება .

In [ ]:
total_calories = []
total_carbs = []
total_fat = []
total_protein = []
total_sodium = []
total_sugar = []
total_fiber = []


totals = [total_calories,total_carbs,total_fat,total_protein,
          total_sodium, total_sugar,total_fiber,]

In [ ]:
def count_totals(tot_data):
    result = {}

    for curr_tot in tot_data:
        prot_name = curr_tot['name']
        if prot_name in not_needed_prots:
            continue
        prot_val = curr_tot['value']
        if prot_name not in result.keys():
            result[prot_name] = prot_val
        else: 
            result[prot_name] += prot_val
        
        totals[prot_names.index(prot_name)].append(prot_val)
        
    return result

ასევე ბარემ ამოვიღოთ გოლები რაც დაუწესა აპლიკაციამ მომხმარებელს 

In [ ]:
goal_calories = []
goal_carbs = []
goal_fat = []
goal_protein = []
goal_sodium = []
goal_sugar = []
goal_fiber = []


goals = [goal_calories,goal_carbs,goal_fat,goal_protein,
         goal_sodium,goal_sugar,goal_fiber]

In [ ]:
def fill_goals(goal_data):
    for curr_goal in goal_data:
        prot_name = curr_goal['name']
        if prot_name in not_needed_prots:
            continue
        prot_value = curr_goal['value']
        goals[prot_names.index(prot_name)].append(prot_value)

### ამათი დანიშნულებაც აღვწეროთ რომ გავერკვევით

In [ ]:
divergence = []
dishes_count = []

In [ ]:
def replacer(value):
    return value.replace(',', '')

In [ ]:
def count_sums():
    result = {}
    
    dishes = json.loads(data.iloc[i,2])
    
    for dish in dishes:
        dish_per_day = dish['dishes']
        dishes_count.append(len(dish_per_day))
        
        for food in dish_per_day:
            nutritions = food['nutritions']
            
            for prot in nutritions:
                prot_name = prot['name']
                if prot_name in not_needed_prots:
                    continue
                prot_val = prot['value']
                if prot_name not in result.keys():
                    result[prot_name] = int(replacer(prot_val))
                else:
                    result[prot_name] += int(replacer(prot_val))
    
    return result

In [ ]:
def checkForDivergence(tots, index):
    sums = count_sums()
    if tots != sums:
        divergence.append((index, sums, tots))

In [ ]:
sum_check = True
gt_check = False
for i in range(data.shape[0]):
    sums = count_sums()

    main_info = json.loads(data.iloc[i,3])

    total_data = main_info['total']     
    goal_data = main_info['goal']
    fill_goals(goal_data)
    
    checkForDivergence(count_totals(total_data), i)


In [ ]:
print(len(divergence))
for index in range(20):
    print(divergence[index])

დივერგენციების სტატუსის შემოწმებისას აღმოჩნდა , რომ 918 ჩანაწერში გვხდება აცდენა ტოტალსა და ხელით აჯამულ მონაცემებს შორის . საინტერესო ფაქტია ის , რომ აცდენილი მნიშვნელობების შეფარდება ზუსტად 2_ის ტოლია. ეს ფაქტი სავარაუდოდ აპლიკაციის ბექენდის გამოა, სადაც მონაცემების გადუბლირება ხდება.  

ამასთანავე, დატის დათვალიერებისას , თვალში მოგვხვდა ზოგიერთი მონაცემის უარყოფითობა . გადავწყვიტეთ , რომ გამოვიკვლიოთ მსგავის უარყოფითი მნიშვნელობები 

In [ ]:
negatives = {}
for i in range(data.shape[0]):
    dcol = json.loads(data.iloc[i,2])
    for dish in dcol:
        for food in dish['dishes']:
            for nutrients in food['nutritions']:
                if int(replacer(nutrients['value'])) <0 :
                    if food['name'] not in negatives.keys():
                        protValTupla = (nutrients['name'],int(replacer(nutrients['value'])))
                        protSet = set(protValTupla)
                        negatives[food['name']] = [1,protSet]
                    else:
                        negatives[food['name']][0] += 1
                        protValTupla = (nutrients['name'],int(replacer(nutrients['value'])))
                        negatives[food['name']][1].add(protValTupla)

In [ ]:
sortedNegatives = sorted(negatives.items(), key=lambda item: -item[1][0])
for key,value in sortedNegatives:
    print(key,value)


In [ ]:
sortedNegatives = sorted(negatives.items(), key=lambda item: -item[1][0])
negativesDict = {'Breastfeeding':0,'CUSTOM_ADJUST':0,
                 'Other':0}
for key,value in sortedNegatives:
    if 'Breastfeeding' in key or 'Generic' in key :
        negativesDict['Breastfeeding'] +=1
    elif 'CUSTOM ADJUST' in key or 'Net Carb Adjustment' in key:
        negativesDict['CUSTOM_ADJUST'] +=1    
    else:
        negativesDict['Other'] +=1
print(negativesDict)
   


In [ ]:
plt.figure(figsize=(20,8))
plt.pie(negativesDict.values(), labels = negativesDict.keys(),autopct = "%1.1f%%")
plt.show()

უარყოფითი დატის გაანალიზებისას , უპირატესობა დაიკავა BreastFeeding_მა. მიუხედავად ამ ჩარტში 22 პროცენტისა გარკვეული მომხმარებლები , BreastFeeding_ს სხვადსხვა ტიპის ქვეშ აერთიანებენ . Other ნაწილი წვრილ-წვრილი და მრავალი ტიპის საკვებია ,ამიტომაც გამოვიდა ყველაზე ბევრი . 

In [ ]:
for goal in goals:
    print(len(goal))

საბოლოოდ , არსებული მონაცემებიდან გვინდა რომ მივიღოთ, უფრო გამარტივებული და 
გასუფთავებული მონაცემები.
ვაგდებთ  ნუტრიენტებიდან 10 ცალ,ყველაზე ცოტა რაოდენობის, ნუტრიენტს.
ახალ ცხრილში გვინდა რომ მონაცემები დავალაგოთ თარიღების ჭრილში.
ნუტრიენტების თითოეული მონაცემი  იქნება ტოტალური მაჩვენებელი,საშუალო მაჩვენებელი დღეების ჭრილში. 


ავაგოთ ახალი დიქშენერი ინფორმაციის . პირველი 3 სვეტი იქნება თარიღი , აქტივობა და რამდენი საჭმელი შეიჭამა იმ დღეს.
შემდეგ იქნება ჯამური და საშუალო რაოდენობები ნუტრიენტების რათა შემდეგ მოხდეს მათი გამოყენება თარიღების კლასტერიზაციისას. თარიღების კლასტერიზაცია იმიტომ ავირჩიეთ, რომ ყველა თარიკი უნიკალურია , და შეიცავს გასაგებ მონაცემებს. // აქ ტექსტი უნდა დავარედაქტიროთ.

In [ ]:
new_data = {}
new_data['Date'] = []
new_data['Week_Day'] = []
new_data['Month'] = []
new_data['User_Activity'] = []
new_data['Food_Count'] = []

In [ ]:
new_data['total_calories'] = []
new_data['total_carbs'] = []
new_data['total_fat'] = []
new_data['total_protein'] = []
new_data['total_sodium']= []
new_data['total_sugar'] = []
new_data['total_fiber'] = []

totals_new_data = [
    new_data['total_calories'], new_data['total_carbs'], new_data['total_fat'],
    new_data['total_protein'], new_data['total_sodium'], new_data['total_sugar'],
    new_data['total_fiber']
]

In [ ]:
for i in range( int((end-start).days)+1):
    dat = start+timedelta(i)
    new_data['Work Day'].append(dat.strftime('%A'))
    new_data['Month'].append(dat.strftime("%B"))

ნუტრიენტების ჯამური რაოდენობების დასათვლელად მოცემულ დატაში გადავყვებით თითოეული საჭმლის მონაცემებს ყოველი თარიღისთვის და ვჯამავთ .

In [ ]:
def countTotals(tmp_data, tmp):   
    for j in range(tmp_data.shape[0]): 

        dloc = json.loads(tmp_data.iloc[j, 2])
        for dish in dloc:
            for food in dish['dishes']:
                for nutrient in food['nutritions']:
                    if nutrient['name'] in prot_names:
                        index = prot_names.index(nutrient['name'])
                        value = replacer(nutrient['value'])
                        tmp[index] += int(value)

In [ ]:
new_data['goal_calories'] = []
new_data['goal_carbs'] = []
new_data['goal_fat'] = []
new_data['goal_protein'] = []
new_data['goal_sodium']= []
new_data['goal_sugar'] = []
new_data['goal_fiber'] = []

goals_new_data = [
    new_data['goal_calories'], new_data['goal_carbs'], new_data['goal_fat'],
    new_data['goal_protein'], new_data['goal_sodium'], new_data['goal_sugar'],
    new_data['goal_fiber']
]

In [ ]:
def countGoals(tmp_data, tmp):   
    for j in range(tmp_data.shape[0]): 

        gloc = json.loads(tmp_data.iloc[j, 3])
        for goal in gloc['goal']:
            if goal['name'] in prot_names:
                name = goal['name']
                index = prot_names.index(name)
                value = goal['value']
                tmp[index] += int(value)

In [ ]:
def countFood(data):
    res = 0
    
    for j in range(data.shape[0]):
        food_entries = json.loads(data.iloc[j, 2])
        food = food_entries[0]
        res += len(food['dishes'])
    
    return res

გადავურბინოთ თითოეულ თარიღს მოცემულ ინტერვალში და დავთვალოთ საჭმლის რაოდენობა, ჯამური ნუტრიენტების რაოდენობები. 

In [ ]:
time_interval = int((end-start).days)+1
for i in range(time_interval):
    date = start+timedelta(i)
    date = date.strftime('%Y-%m-%d')
    
    tmp_data = data[data[data.columns[1]] == date]
    food_count = countFood(tmp_data)
    
    tmp_totals = np.zeros(len(prot_names))
    countTotals(tmp_data, tmp_totals)
    for k in range(len(prot_names)):
        totals_new_data[k].append(tmp_totals[k])
        
    tmp_goals = np.zeros(len(prot_names))
    countGoals(tmp_data, tmp_goals)
    for k in range(len(prot_names)):
        goals_new_data[k].append(tmp_goals[k])
        
    new_data['Date'].append(date)
    new_data['User_Activity'].append(len(tmp_data[tmp_data.columns[0]].unique()))
    new_data['Food_Count'].append(food_count)
    
    

In [ ]:
frame = pd.DataFrame(new_data)
frame

# Sesacvlelia

საბოლოოდ მივიღეთ ახალი ცხრილი , რომელშიც 500000 სტრიქონის მაგივრად გვაქვს 208 სტრიქონი და 2გბ_ს მაგივრად რამდენიმე კილობაიტი ინფორმაცია . საინტერესო ფაქტი , რომელიც ვერ გავარკვიეთ ჯერ არის , რომ დატაში სრულიად უგულვებელყოფილია სოდიუმის ასპექტი . რატომ არ ვიცით.თუმცა ვიკვლევთ.

## მიზანსა და რეალობას შორის სხვაობა

In [ ]:
new_data['diff_calories'] = []
new_data['diff_carbs'] = []
new_data['diff_fat'] = []
new_data['diff_protein'] = []
new_data['diff_sodium']= []
new_data['diff_sugar'] = []
new_data['diff_fiber'] = []

for i in frame.iterrows():
    for prot in prot_names:
        tot = 'total_' + prot.lower()
        goal = 'goal_' + prot.lower()
        df = 'diff_' + prot.lower()
        diff = i[1][tot]-i[1][goal]
        new_data[df].append(diff)
    

In [ ]:
tmp_frame = pd.DataFrame(new_data)
tmp_frame

## შევინახოთ ცალკე ფაილად და განვაგრძოთ და დავტესტოთ რომ ინახება, ვაშა მზადაა კლასტერიზაციისთვის, თუ დაგვჭირდა average-ებს იქაც დავთვლით

In [ ]:
tmp_frame.to_csv('DataForClusterization.csv')
testDB = pd.read_csv('DataForClusterization.csv')
testDB